# BUSS6002 Week 10: Task 1

In [1]:
import pandas as pd

# Load in data
airline = pd.read_csv('airline_small.csv', encoding='ISO-8859-1')

# Convert the minutes of delay into either a "delay" (1) or "not delay" (0)
airline['Delayed'] = (airline['ArrDelay'] > 20).astype(int)

# Just grab these two columns: Distance is the feature, and Delayed is the target
sub_airline = airline[['Distance', 'Delayed']]
# Drop any rows with missing data
sub_airline = sub_airline.dropna()

sub_airline.head()

C:\Program Files\Anaconda3-5.0.1x64\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (10,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Distance,Delayed
0,308.0,0
1,296.0,0
2,480.0,1
3,296.0,0
4,373.0,0


# Create our Mapper and Reduce functions

In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

# Again, every map function is applied just on a chunk of data
# In this map function, we will try to calculate the X^T X and X^T y for that chunk
def map_linear_fn(chunk):
    # Get the dependant variable y
    y = chunk['Delayed'].values

    # Get the independant/feature variables
    # which is everything except the price column
    x = chunk['Distance'].values.reshape(-1, 1)

    # Initialize our model
    model = LogisticRegression()
    
    # Fit your model
    model.fit(x, y)
    
    # Grab the actual parameters
    parameters = np.append(np.array(model.intercept_), model.coef_)
    
    # Yield the result
    yield("result", parameters)

# The reduce function should add up all the X^T X and all the X^T y separately,
# and then, it should do the final (X^T X)^{-1} X^T y calculation for the linear regression weights
def reduce_linear_fn(key, values):

    # Store all the resulting parameters from each map worker into a list
    parameter_list = []
    for parameter in values:
        parameter_list.append(np.array(parameter))
        
    # Average all the parameters (over axis 0 which is over the chunks)
    average_parameter = np.mean(parameter_list, axis=0)

    yield (key, average_parameter)

In [9]:
from mockr import run_pandas_job

results = run_pandas_job(sub_airline, map_linear_fn, reduce_linear_fn, n_chunks=2)

print(results)

[('result', array([-1.80855714e+00,  1.39461583e-04]))]
